<img style="float: left;" alt="Drawing" src="./figures/3Di_beeldmerk_RGB.png" width="100"/>

## Starting a 3Di simulation with control structures

Welcome! In this notebook we will show you how to start a <a href="https://3diwatermanagement.com/">3Di</a>-simulation in a jupyter notebook by using the API-v3. In addition, we will show you how to set control structures.

The example we use sets the discharge coefficient in two orifices to almost zero based on a water level at a certain connection node. This is an example of the use of control structures. 3Di knows various control structures: timed control, table control, & memory control. For more detail check out the documentation: https://docs.3di.live/c_control.html 

The following steps will be taken according to an example of an 3Di model:
- **step 1:** Creating a 3Di-simulation by using the threedi-api
- **step 2:** Adding events to this 3Di-simulation including control structures by using the threedi-api
- **step 3:** Running the 3Di-simulation by using the threedi-api

**Step 1: Starting a 3Di-simulation by using the threedi-api**

Importing all required packages:

In [ ]:
from datetime import datetime
from getpass import getpass

from openapi_client import ApiException, SimulationsApi, OrganisationsApi
from openapi_client.api import AuthApi
from openapi_client.api import ThreedimodelsApi
from openapi_client.models import RasterEdit
from threedi_api_client.threedi_api_client import ThreediApiClient


Provide your credentials to connect to the threedi-api:

In [ ]:
api_host = "https://api.3di.live/v3.0"
username = input("Username: ")
password = getpass("Password: ")
config = {
    "API_HOST": api_host,
    "API_USERNAME": username,
    "API_PASSWORD": password
}

api_client = ThreediApiClient(config=config)

Check the connection with your provided credentials:

In [ ]:
auth_api = AuthApi(api_client)

try:
    user = auth_api.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

1.3  In order to run a simulation you need a threedi-model. Let's see which threedi-models are available:

In [ ]:
models_api = ThreedimodelsApi(api_client)

part_of_the_model_name = "aleida"

#help(models_api.threedimodels_list) # Use this line to see what options for filtering are available

models = models_api.threedimodels_list(name__icontains=part_of_the_model_name, limit = "20")  # limit to the first 20 results
for model in models.results:
    print(f"{model.name, model.id}")
    
#print(models)
   


In [ ]:
#set the model id of the model to use in this calculation
model_id = 24158

In this notebook we will use a sample model with control structures in it

Now that we have a model we are almost ready to create the simulation. However, first we'll need to get an organisation under which's name we will run the simulation.

Let's see which organisations are available within my user account:

In [ ]:
organisation_api = OrganisationsApi(api_client)

organisations = organisation_api.organisations_list(name__icontains="N&S")

for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

In this example we use the organisation from Nelen & Schuurmans:

organisation_uuid = "61f5a464c35044c19bc7d4b42d7f58cb"

In [ ]:
#organisation_uuid = "61f5a464c35044c19bc7d4b42d7f58cb" #Nelen & Schuurmans
organisation_uuid = "b08433fa47c1401eb9cbd4156034c679" #N&S IT

1.5 Let's create the simulation of the chosen model now, with this organisation uuid. Note that it will not run yet.

In [ ]:
simulation_api = SimulationsApi(api_client)

name_for_simulation = "demo_simulation_control_structures 2"
sim_duration = 3600

my_simulation = simulation_api.simulations_create(
    data={
        "name": name_for_simulation,
        "threedimodel": model_id,
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": sim_duration  # in seconds, so we simulate for 1 hour
    }
)


#print an overview of the simulation
my_simulation

You can check the status of the simulation with the following api call:

In [ ]:
#check the status of the simulation with:
status = simulation_api.simulations_status_list(my_simulation.id)
print(status)

We can see the simulation has not started yet. The options at the name of the status can be: "created", "started" and "finished".

**Step 2: Adding events to this 3Di-simulation by using the threedi-api**

In the previous step we created a simulation for the 3Di model of rockflow. Several events can be added to this 3Di-simulation:

* initial waterlevels
* rain
* breaches
* laterals
* control structures 


In this step we create a simple constant rain event of 30 minutes with an offset of 5 minutes:

In [ ]:
from openapi_client.models import ConstantRain
constant_rain = ConstantRain(
    simulation=my_simulation.id,   # the ID we got from our create call above
    offset=300,        # let the rain start after 5 minutes
    duration=1800,    # let the rain last for half an hour
    value=0.0000278,     # in m/s, 0.0000278 = 100 mm/h
    units="m/s"       # the only unit supported for now
)

print (constant_rain)

Now we add this constant rain event to our created simulation

In [ ]:
simulation_api.simulations_events_rain_constant_create(my_simulation.id, constant_rain)

2.3 We can get an overview of the added events to our 3Di-simulation by the following api-call:

In [ ]:
events = simulation_api.simulations_events(my_simulation.id)
print(events)

So, we can indeed see here that we have only added the constant rain event to our 3di-simulation.

**Step 2.2. Adding controls to the simulation**

In [ ]:
#help(simulation_api) #use this line to see all options available
help(simulation_api.simulations_events_structure_control_table_create)  # we use this one the request information on the endpoint

In [ ]:
# As we can see we need to provide data on the control. 

from openapi_client.models import MeasureSpecification
from openapi_client.models import MeasureLocation
from openapi_client.models import TableStructureControl

In [ ]:
# First we define the location where the measure is taken. 
measure_location1 = MeasureLocation (
    weight = 1.0,
    content_type = 'v2_connection_node',
    content_pk = 92776
)

# Then we define what is measured and the operator sign. This indicates whether the control has to act when (in this example waterlevel) is higher or lower than a certain value
measure1 = MeasureSpecification (
    name = 'measure point 1',
    locations = [measure_location1], #in case of multiple locations add as a list
    variable = 's1', #waterlevel measurement variable, one of the following options: s1 (waterlevel), vol1 (volume), q (discharge), u1 (velocity)
    operator = '>')

control_in_simulation = TableStructureControl (
    duration = sim_duration,
    offset = 0,
    measure_specification = measure1, #defined before
    structure_id = 5, #this is the id of the structure in the model. 
    structure_type ='v2_orifice', #[ v2_pumpstation, v2_pipe, v2_orifice, v2_culvert, v2_weir, v2_channel ]
    type = 'set_discharge_coefficients', # [ set_discharge_coefficients, set_crest_level, set_pump_capacity ]
    values = [[-2.60,0.0063,0.0063]] #When the water level exceeds the value of -2.60 m MSL the discharge coefficient of this structure is set to 0.063 (almost closed)
)

In [ ]:
simulation_api.simulations_events_structure_control_table_create(my_simulation.id, control_in_simulation)

In [ ]:
# First we define the location where the measure is taken. 
measure_location2 = MeasureLocation (
    weight = 1.0,
    content_type = 'v2_connection_node',
    content_pk = 92776
)

# Then we define what is measured and the operator sign. This indicates whether the control has to act when (in this example waterlevel) is higher or lower than a certain value
measure2 = MeasureSpecification (
    name = 'measure point 2',
    locations = [measure_location2], #in case of multiple locations add as a list
    variable = 's1', #waterlevel measurement variable, one of the following options: s1 (waterlevel), vol1 (volume), q (discharge), u1 (velocity)
    operator = '>')

control_in_simulation2 = TableStructureControl (
    duration = sim_duration,
    offset = 0,
    measure_specification = measure2, #defined before
    structure_id = 6, #this is the id of the structure in the model. 
    structure_type ='v2_orifice', #[ v2_pumpstation, v2_pipe, v2_orifice, v2_culvert, v2_weir, v2_channel ]
    type = 'set_discharge_coefficients', # [ set_discharge_coefficients, set_crest_level, set_pump_capacity ]
    values = [[-2.60,0.0063,0.0063]] #When the water level exceeds the value of -2.60 m MSL the discharge coefficient of this structure is set to 0.063 (almost closed)
)

In [ ]:

simulation_api.simulations_events_structure_control_table_create(my_simulation.id, control_in_simulation2)

**Step 3: Running the 3Di-simulation by using the threedi-api**

We will now start our simulation with the constant rain event:

In [ ]:
simulation_api.simulations_actions_create(my_simulation.id, data={"name": "start"})

We can check the status of the 3Di-simulation with:

In [ ]:
#check the status of the simulation with:
status = simulation_api.simulations_status_list(my_simulation.id)
print(status)


**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
The end

In this tutorial we showed you:
- how to start a simulation of a 3Di-model, 
- how to add a simple rain event to your simulation, 
- how to add control to your simulation
- how to see the progress

And all possible within a Jupyter notebook by making use of the API-v3. Ofcourse there are a lot more options for the analysis possible, by using threedigrid. Or for the events you can add to your simulation. Do you want to learn more or are you interested in some more possible analysis in a jupyter notebook? 

**Please contact**:

Jonas van Schrojenstein | jonas.vanschrojenstein@nelen-schuurmans.nl
